## Métrique originale

Une notion de distance définie comme la norme d'une transformation de (x + I) à (y + I) où I est la matrice identité.

In [1]:
import numpy as np
import time

"""
    Mettre `skiprows=1` pour avoir toutes les données.
    (Mais c'est très long surtout pour faire des tests.)
"""
mnist_train = np.loadtxt('mnist_train.csv', dtype='int', delimiter=',', skiprows=59001)
mnist_test  = np.loadtxt('mnist_test.csv', dtype='int', delimiter=',', skiprows=9751)

train_label = mnist_train[:,0]
train_data  = mnist_train[:,1:]

test_label  = mnist_test[:,0]
test_data   = mnist_test[:,1:]

L'ajout de la matrice identité peut être interprèter ici comme du bruit. Elle sert en fait à ce que les matrices aient un inverse pour pouvoir résoudre $ (B + I)x = Bx + Ix = y $. On cherche $ B $ qui est obtenue avec $ B = (y - x) x^{-1} $.

La norme de la matrice est ici la plus grande valeur propre de la matrice $ B $.

In [2]:
I = np.identity(28)

def metric_4(X, Y):
    x = X.reshape((28,28)) + I
    y = Y.reshape((28,28)) + I
    
    eigv = np.linalg.eigvals((y - x) * np.linalg.inv(x))
    
    eigv = np.log([np.absolute(v) for v in eigv if not (v == 0)] + [1.0])
    
    return np.max(eigv)

def metrics_4(exs):
    s = exs.shape[0]
    r = np.zeros((s,s))
    for i in range(s):
        for j in range(i, exs.shape[0]):
            tmp = metric_4(exs[i], exs[j])
            r[i,j] = tmp
            r[j,i] = tmp
    return r


In [3]:
dis_matrix = metrics_4(train_data)
print(dis_matrix.shape)
#(1000, 1000)

(1000, 1000)


In [4]:
test = metrics_4(test_data)
print(test.shape)
#(250, 250)

(250, 250)


À noter que `metric_4` n'est pas très performante et `metrics_4` à une complexité quadratique. J'ai essayé et abandonné avec 10000 exemples après ~1h00. Par contre, une fois la matrice de similarité calculé, les algorithmes ici-bas sont très rapide.

In [5]:
from sklearn.neighbors import KNeighborsClassifier as knn

# `metric` peut être `callable` ici.
algo = knn(n_neighbors=5, metric='euclidean', n_jobs=6)

algo.fit(train_data, train_label)

result = algo.score(test_data, test_label)

print("Taux de succes avec knn et la distance euclidienne: %.4f %%" % (result * 100.0))

#Taux de succes avec knn et la distance euclidienne: 83.2000 %

Taux de succes avec knn et la distance euclidienne: 83.2000 %


In [6]:
# `metric` peut être `callable` ici.
algo = knn(n_neighbors=5, metric=metric_4, n_jobs=6)

algo.fit(train_data, train_label)

result = algo.score(test_data, test_label)

print("Taux de succes avec knn et metric_4: %.4f %%" % (result * 100.0))

#Taux de succes avec knn et metric_4: 60.4000 %

Taux de succes avec knn et metric_4: 60.4000 %


On note que c'est moins bon que la distance euclidienne mais le pure hasard serait de 10% alors que nous avons ici 60%.

In [7]:
# Dans la documentation ça semble être un classifieur qui utilise un arbre binaire
# Il sera possible d'y accéder plus tard
from sklearn.cluster import AgglomerativeClustering as BinaryTree

# `affinity` peut être `callable` ici.
# `linkage='average'` voir énoncé, dernier point de Conseils et indications
algo = BinaryTree(n_clusters=10, linkage='average', affinity='precomputed')

pred_labels = algo.fit_predict(dis_matrix)

count_pred = np.zeros((10,10))

for i in range(10):
    for j in range(pred_labels.shape[0]):
        if train_label[j] == i:
            count_pred[i, pred_labels[j]] += 1

print("Distribution des prédictions (classes, prédictions):")
print(count_pred)

# Considérons la classe prédite la plus répéter comme la bonne classe
# (Ne pouvons nous pas toujours faire ça?)
argmax_pred = np.array([np.argmax(count_pred[i]) for i in range(10)])

print("Argmax pour chaque classes:")
print(argmax_pred)

#Distribution des prédictions (classes, prédictions):
#[[ 95.   0.   1.   0.   0.   0.   1.   0.   1.   0.]
# [102.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
# [ 95.   9.   1.   0.   0.   0.   0.   0.   0.   0.]
# [102.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
# [ 93.   1.   0.   0.   0.   1.   0.   1.   0.   0.]
# [ 87.   1.   0.   0.   1.   0.   0.   0.   0.   1.]
# [104.   0.   0.   1.   2.   0.   0.   0.   0.   0.]
# [103.   0.   3.   0.   0.   0.   1.   0.   0.   0.]
# [ 93.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
# [ 98.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
#Argmax pour chaque classes:
#[0 0 0 0 0 0 0 0 0 0]


Distribution des prédictions (classes, prédictions):
[[ 95.   0.   1.   0.   0.   0.   1.   0.   1.   0.]
 [102.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 95.   9.   1.   0.   0.   0.   0.   0.   0.   0.]
 [102.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 93.   1.   0.   0.   0.   1.   0.   1.   0.   0.]
 [ 87.   1.   0.   0.   1.   0.   0.   0.   0.   1.]
 [104.   0.   0.   1.   2.   0.   0.   0.   0.   0.]
 [103.   0.   3.   0.   0.   0.   1.   0.   0.   0.]
 [ 93.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 98.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
Argmax pour chaque classes:
[0 0 0 0 0 0 0 0 0 0]


La même classe est prédite à chaque fois. On pourrait donc parler de hasard et approximer cela à 10%.

On note aussi quelque exemples qui selon l'algorithme ne ressemble à rien puisqu'ils sont seuls dans leur classes.

In [9]:
# Dans la documentation ça semble être un classifieur qui utilise un arbre binaire
# Il sera possible d'y accéder plus tard
from sklearn.cluster import AgglomerativeClustering as BinaryTree

# `affinity` peut être `callable` ici.
# `linkage='average'` voir énoncé, dernier point de Conseils et indications
algo = BinaryTree(n_clusters=10, linkage='average', affinity='euclidean')

pred_labels = algo.fit_predict(train_data)

count_pred = np.zeros((10,10))

for i in range(10):
    for j in range(pred_labels.shape[0]):
        if train_label[j] == i:
            count_pred[i, pred_labels[j]] += 1

print("Distribution des prédictions (classes, prédictions):")
print(count_pred)

# Considérons la classe prédite la plus répéter comme la bonne classe
# (Ne pouvons nous pas toujours faire ça?)
argmax_pred = np.array([np.argmax(count_pred[i]) for i in range(10)])

print("Argmax pour chaque classes:")
print(argmax_pred)

#Distribution des prédictions (classes, prédictions):
#[[  0.   2.   3.   0.   3.  81.   2.   0.   7.   0.]
# [  0. 102.   0.   0.   0.   0.   0.   0.   0.   0.]
# [  0.  18.   0.   0.  78.   1.   0.   8.   0.   0.]
# [  0.  76.  12.   0.  15.   0.   0.   0.   0.   0.]
# [  1.  83.   0.   0.   0.   0.   0.   0.   0.  12.]
# [  0.  75.  15.   0.   0.   0.   0.   0.   0.   0.]
# [  0. 107.   0.   0.   0.   0.   0.   0.   0.   0.]
# [  0.  89.   0.  17.   0.   0.   1.   0.   0.   0.]
# [  1.  80.  12.   0.   0.   1.   0.   0.   0.   0.]
# [  0.  86.   0.   0.   0.   0.   0.   0.   0.  12.]]
#Argmax pour chaque classes:
#[5 1 4 1 1 1 1 1 1 1]


Distribution des prédictions (classes, prédictions):
[[  0.   2.   3.   0.   3.  81.   2.   0.   7.   0.]
 [  0. 102.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  18.   0.   0.  78.   1.   0.   8.   0.   0.]
 [  0.  76.  12.   0.  15.   0.   0.   0.   0.   0.]
 [  1.  83.   0.   0.   0.   0.   0.   0.   0.  12.]
 [  0.  75.  15.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 107.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  89.   0.  17.   0.   0.   1.   0.   0.   0.]
 [  1.  80.  12.   0.   0.   1.   0.   0.   0.   0.]
 [  0.  86.   0.   0.   0.   0.   0.   0.   0.  12.]]
Argmax pour chaque classes:
[5 1 4 1 1 1 1 1 1 1]


Ça n'est pas vraiment meilleur qu'avec la distance originale.

Je me demande si les quelques exemples seuls dans leur classes ne seraient pas des observations abérantes du jeu de données. En d'autres mots, la distance varie énormément sur quelques données et très peu sur le reste.

Est-ce que les données abérantes pourraient faussé les résultats d'un algorithme de regroupement?